In [5]:
%%file Q1a.py
import numpy as np
import math
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size
A = [[1,1,0,3] ,[2,1,-1,1] ,[3,-1,-2,2] ,[-1,2,3,-1]]

n = len(A)
# Pset is the row a cpu stores
if rank ==0:
    Pset = {0,2}
if rank ==1:
    Pset = {1,3}
numP = 2
# start LU decomposition
for k in range(n):
    if k in Pset:
        # send the data of current cpu to the rest of cpus
        for p in range(numP):
            if p!= rank:
                comm.send(A[k], dest=p, tag=11)
    else:
        A[k] = comm.recv(source=k%numP, tag=11)
    for i in range(k+1,n):
        if i in Pset:
            A[i][k] = A[i][k]/A[k][k]
            for j in range(k+1, n):
                A[i][j] = A[i][j]-A[i][k]*A[k][j]
if rank ==0:
    print(str(A[0])+'\n'+str(A[1])+'\n'+str(A[2])+'\n'+str(A[3])+'\n')

Overwriting Q1a.py


In [6]:
!mpiexec -np 2 python Q1a.py

[1, 1, 0, 3]
[2.0, -1.0, -1.0, -5.0]
[3.0, 4.0, 2.0, 13.0]
[-1.0, -3.0, 0.0, -13.0]

